In [13]:

from dotenv import load_dotenv
load_dotenv()
import os
import sys
sys.path.append(
    os.environ.get('WORK_DIR'))
sys.path.append(
    os.environ.get('DBASE_DIR'))
from dbase.DataAPI.ThetaData import *
from dbase.database.SQLHelpers import *
import pandas as pd
from data import HistoricTradeDataHandler
from event import *
# from strategy import OptionSignalStrategy
# from portfolio import OptionSignalPortfolio
from queue import Queue
from trade.backtester_.backtester_ import PTDataset, PTBacktester
import pandas_ta as ta
from trade.assets.Stock import Stock
from trade.backtester_.utils.WalkForwardUtils import prev_monday
from trade.backtester_.strats import BBandsTrend2
import yfinance as yf

from datetime import datetime
from backtesting.backtesting import Strategy as BacktestingStrategy
%load_ext autoreload
%autoreload 2

In [2]:
contracts = list_contracts('AAPL', '20231103')
contracts

,root,expiration,strike,right
0,AAPL,20240119,260.0,C
1,AAPL,20231117,129.0,P
2,AAPL,20241220,145.0,C
3,AAPL,20241220,145.0,P
4,AAPL,20240119,80.0,P
...,...,...,...,...
1093,AAPL,20231124,145.0,P
1094,AAPL,20231215,145.0,P
1095,AAPL,20231215,145.0,C
1096,AAPL,20240216,145.0,P


In [31]:
querystring = {"root":"AAPL","exp":"20231103","strike":"170000","right":"C","start_date":"20231001","end_date":"20231103","ivl":"3600000", "use_csv": "true"}

option = retrieve_option_ohlc('AAPL', '20231103', 170.0, 'C', "20231001", "20231103")
option

,ms_of_day,open,high,low,close,volume,count,date
5,52200000,7.80,7.80,7.80,7.80,10,3,2023-10-02
11,52200000,7.61,7.66,7.61,7.65,5,5,2023-10-03
17,52200000,7.90,8.35,7.90,8.25,231,16,2023-10-04
23,52200000,9.15,9.20,8.98,9.05,14,9,2023-10-05
29,52200000,10.57,10.85,10.50,10.85,6,5,2023-10-06
35,52200000,11.10,11.40,11.10,11.20,33,6,2023-10-09
41,52200000,11.04,11.04,11.04,11.04,1,1,2023-10-10
47,52200000,11.30,11.30,11.30,11.30,1,1,2023-10-11
53,52200000,11.73,12.25,11.73,12.21,3,3,2023-10-12
59,52200000,11.50,11.80,11.50,11.80,16,3,2023-10-13


In [30]:
date = pd.to_datetime('2023-11-04')
closest_index = (option.index - date).to_series().abs().argsort()[:1]
print(closest_index)
closest = option.iloc[closest_index]
type(closest.iloc[0])

date
-33 days    24
Name: date, dtype: int64


pandas.core.series.Series

In [4]:
data = option.iloc[0]
data

ms_of_day    52200000
open              7.8
high              7.8
low               7.8
close             7.8
volume             10
count               3
date         20231002
Name: 5, dtype: object

In [6]:
data['close'] * 100

780.0

In [9]:
import pandas as pd

# Sample DataFrame with datetime
data = {'date': pd.to_datetime(['2023-10-01', '2023-10-10', '2023-10-20', '2023-10-30'])}
df = pd.DataFrame(data)

# Target datetime
target = pd.to_datetime('2023-10-10')

# Find the closest datetime
closest_row = df.iloc[(df['date'] - target).abs().argsort()[:1]].iloc[0]

print(type(closest_row))

<class 'pandas.core.series.Series'>


In [47]:
import pandas as pd
import numpy as np

# Sample DataFrame
data = {
    'A': [1, 1.0, None, 3, 1.0],
    'B': [None, 1.0, 5, 0.0, 0.0],
    'C': [0.0, 7, 7, 7, 1]
}
df = pd.DataFrame(data)

# Count rows with any empty (NaN) values
empty_rows = df.isnull().any(axis=1).sum()

# Count rows where all values are 0.0
zero_rows = (df == 0.0).any(axis=1).sum()

# Count valid rows (not empty and not all zeros)
valid_rows = len(df) - (empty_rows + zero_rows)

# Compare counts
if (empty_rows + zero_rows) > valid_rows:
    print("More rows with empty or 0.0 than valid data.")
else:
    print("More valid rows than empty or 0.0 rows.")
    
print('empty: ',empty_rows)
print('zero: ',zero_rows)
print('valid: ',valid_rows)

More rows with empty or 0.0 than valid data.
empty:  2
zero:  3
valid:  0


In [ ]:
lo = 0
lo += 1
lo

In [34]:
import pandas as pd

# Sample DataFrame with datetime as index
data = {'value': [1, 2, 3, 4]}  # Sample data values for illustration
dates = pd.to_datetime(['2023-10-01', '2023-10-10', '2023-10-20', '2023-10-30'])
df = pd.DataFrame(data, index=dates)

# Target datetime
target = pd.to_datetime('2023-10-15')

# Find the closest datetime
closest_index = (df.index - target).to_series().abs().argsort()[:1]
closest_row = df.iloc[closest_index]

print(type(closest_row.iloc[0]))

<class 'pandas.core.series.Series'>


In [44]:
import requests
from io import StringIO
def retrieve_option_ohlc_mod(symbol: str, exp:str, strike : float, right:str, start_date:str, end_date:str ): 
    """
        returns eod ohlc for all the days between start_date and end_date 
        Interval is default to 3600000
    """
    strike = strike * 1000
    strike = int(strike) if strike.is_integer() else strike
    url = "http://127.0.0.1:25510/v2/hist/option/ohlc"
    querystring = {"end_date": end_date, "root": symbol, "use_csv": "true", "exp": exp, "ivl": 3600000, "right": right, "start_date": start_date, "strike": strike}
    headers = {"Accept": "application/json"}
    response = requests.get(url, headers=headers, params=querystring)
    if(__isSuccesful(response.status_code)): 
        data = pd.read_csv(StringIO(response.text))
        if (len(data.columns)) > 1: 
            data['mean_volume'] = data.groupby('date')['volume'].transform('mean')
            data = data.loc[data.groupby('date')['volume'].apply(lambda x: (x - x.mean()).abs().idxmin())]
            data = data.drop_duplicates(subset='date', keep='last')
            data = data.drop(columns=['mean_volume'])
            data['date'] = pd.to_datetime(data['date'], format='%Y%m%d')
            return data
        else: 
            print('Error in retrieving data: ', data) 
            return f"No data retrieved {response.status_code}  {response.text}"
    else: 
        return f"{response.status_code}  {response.text}"
    
def __isSuccesful(status_code: int): 
    return status_code >= 200 and status_code < 300

In [45]:
option = retrieve_option_ohlc_mod('AMZN', '20241018', 200.0, 'C', "20230304", "20240524")
pd.set_option('display.max_rows', None)    # Show all rows
pd.set_option('display.max_columns', None)  # Show all columns
pd.set_option('display.expand_frame_repr', False)  # Prevent line wrapping
print(option)

     ms_of_day   open   high    low  close  volume  count       date
0     34200000   0.00   0.00   0.00   0.00       0      0 2024-02-07
6     34200000   0.00   0.00   0.00   0.00       0      0 2024-02-08
12    34200000   0.00   0.00   0.00   0.00       0      0 2024-02-09
18    34200000   9.12   9.12   9.12   9.12       2      1 2024-02-12
24    34200000   0.00   0.00   0.00   0.00       0      0 2024-02-13
30    34200000   0.00   0.00   0.00   0.00       0      0 2024-02-15
36    34200000   0.00   0.00   0.00   0.00       0      0 2024-02-16
43    37800000   0.00   0.00   0.00   0.00       0      0 2024-02-20
52    48600000   7.21   7.21   7.21   7.21       1      1 2024-02-21
56    41400000   8.87   8.87   8.87   8.87       1      1 2024-02-22
60    34200000   9.70   9.70   9.70   9.70       1      1 2024-02-23
67    37800000   9.65   9.65   9.47   9.47       2      2 2024-02-26
75    45000000   8.65   8.65   8.65   8.65       3      1 2024-02-27
79    37800000   0.00   0.00   0.0

In [42]:
from IPython.display import display
option = retrieve_option_ohlc_mod('AMZN', '20241018', 200.0, 'C', "20230304", "20240524")
pd.set_option('display.max_rows', None)    # Show all rows
pd.set_option('display.max_columns', None)  # Show all columns
pd.set_option('display.expand_frame_repr', False)  # Prevent line wrapping
print(option)

     ms_of_day   open   high    low  close  volume  count       date
0     34200000   0.00   0.00   0.00   0.00       0      0 2024-02-07
1     37800000   0.00   0.00   0.00   0.00       0      0 2024-02-07
2     41400000   0.00   0.00   0.00   0.00       0      0 2024-02-07
3     45000000   0.00   0.00   0.00   0.00       0      0 2024-02-07
4     48600000   7.52   7.52   7.52   7.52      10      1 2024-02-07
5     52200000   0.00   0.00   0.00   0.00       0      0 2024-02-07
6     34200000   0.00   0.00   0.00   0.00       0      0 2024-02-08
7     37800000   7.80   7.80   7.80   7.80       2      1 2024-02-08
8     41400000   0.00   0.00   0.00   0.00       0      0 2024-02-08
9     45000000   0.00   0.00   0.00   0.00       0      0 2024-02-08
10    48600000   0.00   0.00   0.00   0.00       0      0 2024-02-08
11    52200000   0.00   0.00   0.00   0.00       0      0 2024-02-08
12    34200000   0.00   0.00   0.00   0.00       0      0 2024-02-09
13    37800000   0.00   0.00   0.0